In [2]:
import pickle
import os
import numpy as np
import tensorflow as tf

import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.slim as slim
import tensorflow.nn as nn

In [3]:
input_dir = 'data/pickles/'

In [5]:
label_2_num_mapping = {}
num_2_label_ar= []
data = {"x":[],"y":[]}

label_num = 0

for label in os.listdir(input_dir):
    if label == '.DS_Store' or label == '.ipynb_checkpoints':
        continue
    p_file = input_dir + label
    label_2_num_mapping[label]=label_num
    num_2_label_ar.append(label)

    with open(p_file, 'rb') as f:
        label_examples  = pickle.load(f)
        # data dim = num_examples * time * frequency
        label_ar = [np.swapaxes(np.array(example),0,1) for example in label_examples]
        data['x'].extend(label_ar)
        y_labels = [label_num for i in range(0,len(label_examples))]
        data['y'].extend(y_labels)
   
    label_num+=1
          

# Samping function

In [6]:
def sample_batch(n, X,Y):
    """
    takes input and returns padded sample
    n= num_samples
    X = input_featurs_list
    Y = label_list
    """
    label_ids = np.random.randint(low = 0,high = len(X), size=n)
    frequency = len(X[0][0])
    sampled_X = [X[label_id] for label_id in label_ids]
    sampled_y = [Y[label_id] for label_id in label_ids]
    padded_X = []
    
    max_batch_len = max([len(x) for x in sampled_X])
    for x in sampled_X:
        padding_time_count = max_batch_len-len(x)
        if padding_time_count!=0:
            x_padded = np.zeros(shape = (max_batch_len,frequency))
            x_padded[:x.shape[0],:x.shape[1]] = x
            padded_X.append(x_padded)
        else:
            padded_X.append(x)
        
    return np.asarray(padded_X),np.asarray(sampled_y)

In [97]:
# x_samples ,y_samples = sample_batch(32,X = data['x'],Y=data['y'])
# print(x_samples.shape)

# Model

In [9]:
class Word_Predictor_RNN():
    def __init__(self, input_freq=20,hidden_size=256,linear_output_size = 128, n_categories=30):
            super().__init__()
            self.rnn = rnn.BasicLSTMCell(hidden_size) 

            input = tf.placeholder(tf.float32, hidden_size, name='output')
            self.linear1 = slim.fully_connected(input, linear_output_size, activation_fn=nn.relu6)

            input = tf.placeholder(tf.float32, linear_output_size, name='categorize')
            self.linear2 = slim.fully_connected(input, n_categories, activation_fn=None)
    def forward(self, x, initial_states):
        
        x = tf.reshape(x,[-1])

        x, final_states = rnn.static_rnn(self.rnn, x, dtype=tf.float32)


# Model  Hyper Parameters

In [8]:
n_categories = len(label_2_num_mapping)
batch_size = 1
input_freq = 20
hidden_size=256
n_epochs = 10
n_iters = 50

# Model Training

In [10]:
model = Word_Predictor_RNN(input_freq=input_freq,hidden_size=hidden_size,n_categories=n_categories)

ValueError: Input 0 of layer fully_connected is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: [256]

In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    for iter in range(n_iters):
        _inputs,_labels = sample_batch(50,data['x'],data['y'])
        inputs = torch.autograd.Variable(torch.from_numpy(_inputs).float())
        targets = torch.autograd.Variable(torch.from_numpy(_labels))

        outputs = model(inputs, None)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        losses[epoch] += loss.data[0]

    if epoch > 0:
        print(epoch, loss.data[0])

    # Use some plotting library
    # if epoch % 10 == 0:
        # show_plot('inputs', _inputs, True)
        # show_plot('outputs', outputs.data.view(-1), True)
        # show_plot('losses', losses[:epoch] / n_iters)

        # Generate a test
        # outputs, hidden = model(inputs, False, 50)
        # show_plot('generated', outputs.data.view(-1), True)

1 3.3885512351989746
2 3.403934955596924


KeyboardInterrupt: 